## Set up

### Package imports

In [1]:
# Import packages
from pathlib import Path
import numpy as np
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence
import pandas as pd
import random
from pdb import set_trace
from sklearn.preprocessing import QuantileTransformer
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt

### Set seed function

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    rs = RandomState(MT19937(SeedSequence(seed)))

### File path

In [3]:
# File path
raw_data_path = '/home/ec2-user/Jul2025/raw_datasets/'
label_path = '/home/ec2-user/Jul2025/labels/'
output_path = '/home/ec2-user/Jul2025/processed_dataset3/'

### Functions

In [4]:
def Gaussian_KDE(df1, df2):
    """
    Function to generate synthetic data using Gaussian KDE.
    Args:
        df1: DataFrame 1 (e.g., Protein/mRNA abundance for Single set)
        df2: DataFrame 2 (e.g., mRNA abundance for Dual set, optional)
        Canvas_Size: Target size for the final dataset (rows x columns)
    Returns:
        final_df: Original data with synthetic data appended
    """
    # Placeholder for synthetic data
    synthetic_data = []

    # Assert df1 and df2 have the same number of columns
    assert df1.columns.size == df2.columns.size

    # Transpose the DataFrames for easier column-wise processing
    df1 = df1.T
    df2 = df2.T

    for _ in range(int((18 * 18 - df1.index.size * 2) / 2)):
        synthetic_row = []
        for col1, col2 in zip(df1.columns, df2.columns):
            # Get the abundance values for the joint distribution
            x = df1[col1].values
            y = df2[col2].values
            
            # Combine x and y for joint KDE
            data = np.vstack([x, y])
            
            # Fit a joint KDE
            kde = gaussian_kde(data)
            
            # Sample from the joint KDE
            sample = kde.resample(1)
            sampled_x, sampled_y = sample[0][0], sample[1][0]
            
            # Add sampled values to the synthetic row
            synthetic_row.append(sampled_x)  # For protein
            synthetic_row.append(sampled_y)  # For mRNA
        
        synthetic_data.append(synthetic_row)

    # Convert synthetic data to a new DataFrame
    print('Synthetic data generated')

    # For Dual, split the synthetic data into two parts
    synthetic_df1 = pd.DataFrame([row[::2] for row in synthetic_data], columns=df1.columns)
    synthetic_df2 = pd.DataFrame([row[1::2] for row in synthetic_data], columns=df2.columns)
    # Combine synthetic with original
    final_df1 = pd.concat([df1, synthetic_df1], axis=0)
    final_df2 = pd.concat([df2, synthetic_df2], axis=0)

    final_df1 = final_df1.T
    final_df2 = final_df2.T

    return final_df1, final_df2

In [5]:
def Normalization(df):

    """
    Normalization across tumors in raw data for protein or mRNA expression.
    """
    
    # Normalize Each Gene Across Tumors 
    df = df.div(df.mean(axis=1),axis=0)

    # Normalize Tumors for Cross-Tumor Comparability
    df = df.div(df.median(axis=0), axis=1)

    # Log2 Transformation
    df = np.log2(df)

    return df

In [6]:
def VisualizeData(PD, MD):
    # Create a figure
    plt.figure(figsize=(12, 12))

    # Create a subplot for PD boxplot
    plt.subplot(2, 2, 1)
    plt.boxplot(PD.values)
    plt.title('Proteome')
    plt.xlabel('Tumors')
    plt.ylabel('Abundances')
    plt.xticks([])

    # Create a subplot for MD boxplot
    plt.subplot(2, 2, 2)
    plt.boxplot(MD.values)
    plt.title('Transcriptome')
    plt.xlabel('Tumors')
    plt.ylabel('Abundances')
    plt.xticks([])

    # Create a subplot for PD histogram
    plt.subplot(2, 2, 3)
    plt.hist(PD.values.flatten(), bins=500)
    plt.title('Proteome')
    plt.xlabel('Tumors')
    plt.ylabel('Count')

    # Create a subplot for MD histogram
    plt.subplot(2, 2, 4)
    plt.hist(MD.values.flatten(), bins=500)
    plt.title('Transcriptome')
    plt.xlabel('Tumors')
    plt.ylabel('Count')

    plt.tight_layout()
    plt.show()

## Create dataframes

### Preparing data for Krug

#### Raw data proceesing

In [7]:
# Open the proteomics data and only keep genes (rows) that are fully quantified
#PFP = 'kr_pro_raw copy.csv' # tumor standardized with mean and std, 2-component Gaussian mixture model-based normalization for gene
#PFP = 'kr_pro_raw.csv' # LethioDDA search: proteomics file (genes table) path, normalized by pool, log2 transformed.
PFP = 'krug prot model use.csv' # MSFragger search, abundances are ratio of channel to pool, no transformation
PD = pd.read_csv(raw_data_path+PFP)

# Indexing and remove NA
PD.index = PD.loc[:,'Index']
PD = PD.loc[:,PD.columns!='Index'] # Shape (12372, 135)
PD.dropna(inplace=True) # Shape (8481, 135)

# Remove tumors that contain keyword 'notinstudy': 3 tumors are not in the study
PD = PD.loc[:,~PD.columns.str.contains('notinstudy')]

# Remove tumors that contain keyword 'Retro': 2 tumors are RetroIR
PD = PD.loc[:,~PD.columns.str.contains('Retro')]

# Find tumors that contain keyword 'REP'
rep_cols = PD.columns[PD.columns.str.contains('REP')]

# Extract tumors' name that has replicates
rep_tumors = set(col.split('.')[0] for col in rep_cols)

# Find columns that match the names in rep_tumors, take average and save it under the tumor name that without REP
for rep_tumor in rep_tumors:
    replicates = PD.columns[PD.columns.str.contains(rep_tumor)]
    PD[rep_tumor] = PD[replicates].mean(axis=1) # Overwrite the original tumor with the average of the replicates
    # Drop the REP column of the rep_tumor only
    replicate_cols = PD.columns[PD.columns.str.contains(rep_tumor+'.REP')]
    PD.drop(replicate_cols, axis=1, inplace=True) # Drop the replicate columns

# Drop tumor X03BR011 because it has no mRNA data
PD.drop('X03BR011', axis=1, inplace=True)

# Sort the column names in the dataframe
PD = PD.reindex(sorted(PD.columns), axis=1) # Shape (8481, 121)

In [8]:
# Open the mRNA data and only keep genes (rows) that are fully quantified
#MFP = 'kr_rna_raw.csv' # mRNA file path, log2 transformed
MFP = 'krug mrna model use.csv' # TPM data, untransformed
MD = pd.read_csv(raw_data_path+MFP)

# Data set wrangling
MD.index = MD.loc[:,'Index']
MD = MD.loc[:,MD.columns!='Index'] # Shape (19938, 121)
MD.dropna(inplace=True) # Shape (19938, 121)

# Drop if any gene that contains 0 read of TPM
MD = MD.loc[~(MD==0).any(axis=1)] # Shape (13627, 121)

# Sort the column names in the dataframe
MD = MD.reindex(sorted(MD.columns), axis=1) # Shape (13627, 121)

#### Normalization

In [9]:
# Visualize the processed data with boxplot and histogram
PD_norm = Normalization(PD)
MD_norm = Normalization(MD)

#### Quantile transform

In [10]:
# Quantile transform each gene
PD_qt = pd.DataFrame(QuantileTransformer(output_distribution='normal',random_state=42).fit_transform(PD_norm), index=PD_norm.index, columns=PD_norm.columns)
MD_qt = pd.DataFrame(QuantileTransformer(output_distribution='normal',random_state=43).fit_transform(MD_norm), index=MD_norm.index, columns=MD_norm.columns)

#### Match gene name in proteome and transcriptome

In [11]:
common_genes = PD_qt.index.intersection(MD_qt.index)
PD_qt = PD_qt.loc[common_genes]
MD_qt = MD_qt.loc[common_genes]

#### Inferencing

In [12]:
# Ensure reproducibility
set_seed(42)

# Synthetic tumors using gaussian KDE
PD_kde, MD_kde = Gaussian_KDE(PD_qt, MD_qt)

Synthetic data generated


#### Filter out genes that has labels

In [13]:
# Open the labels data
LFP = 'markers.txt'
LD = pd.read_csv(filepath_or_buffer=label_path+LFP,sep='\t')

# Data set wrangling
LD.index = LD.loc[:,'Protein']
LD = LD.loc[:,LD.columns!='Protein']

# Remove unclassified class
NotUnclassInd = LD.loc[:,'Localization'] != 'Unclassified'
LD = LD.loc[NotUnclassInd,:]

In [14]:
# Keep only genes (rows) are presented in proteome, mRNA and localization data sets
IntersectingGenes = [value for value in PD_kde.index if ((value in MD_kde.index) & (value in LD.index))]
PD_kde = PD_kde.loc[IntersectingGenes,:]
MD_kde = MD_kde.loc[IntersectingGenes,:]
LD = LD.loc[IntersectingGenes,:]

# Sanity check for the number of genes in each dataframe
print('Krug')
print(len(PD_kde.index))
print(len(MD_kde.index))
print(len(LD.index))

Krug
2693
2693
2693


In [15]:
# Count the number of genes in each localization class
LocalizationCounts = LD.loc[:,'Localization'].value_counts()
print('Number of genes in each localization class:')
print(LocalizationCounts)

Number of genes in each localization class:
Localization
Cytosol         1036
Secretory        769
Nucleus          624
Mitochondria     264
Name: count, dtype: int64


### Preparing data for Mertins

#### Raw data processing

In [16]:
# Mertins data set
# Open the proteomics data and only keep genes (rows) that are fully quantified
#M_PFP = 'me_protein_dropna.csv' # proteomics file path, normalized by pool, log2 transformed.
M_PFP = 'mertins prot model use.csv' # MSFragger search, abundances are ratio of channel to pool, no transformation
M_PD = pd.read_csv(raw_data_path+M_PFP)

# Data set wrangling
M_PD.index = M_PD.loc[:,'Index']
M_PD = M_PD.loc[:,M_PD.columns!='Index'] # Shape (12302, 111)
M_PD.dropna(inplace=True) # Shape (7064, 111)

# Remove tumors that contain keyword 'notinstudy': 28 tumors are not in the study
M_PD = M_PD.loc[:,~M_PD.columns.str.contains('notinstudy')]

# Find tumors that contain keyword 'REP'
M_PD_rep_cols = M_PD.columns[M_PD.columns.str.contains('REP')]

# Extract tumors' name that has replicates
M_PD_rep_tumors = set(col.split('.')[0] for col in M_PD_rep_cols)

# Find columns that match the names in rep_tumors, take average and save it under the tumor name that without REP
for M_PD_rep_tumor in M_PD_rep_tumors:
    M_PD_replicates = M_PD.columns[M_PD.columns.str.contains(M_PD_rep_tumor)]
    M_PD[M_PD_rep_tumor] = M_PD[M_PD_replicates].mean(axis=1) # Overwrite the original tumor with the average of the replicates
    # Drop the REP column of the rep_tumor only
    M_PD_replicate_cols = M_PD.columns[M_PD.columns.str.contains(M_PD_rep_tumor+'.REP')]
    M_PD.drop(M_PD_replicate_cols, axis=1, inplace=True) # Drop the replicate columns

# Sort the column names in the dataframe
M_PD = M_PD.reindex(sorted(M_PD.columns), axis=1) # Shape (7064, 77)

In [17]:
# Open the mRNA data and only keep genes (rows) that are fully quantified
#M_MFP = 'me_rna_dropna.csv' # mRNA file path, gene centric median normalized, log2 transformed
M_MFP = 'mertins mrna model use.csv' # TPM data, untransformed
M_MD = pd.read_csv(raw_data_path+M_MFP)

# Data set wrangling
M_MD.index = M_MD.loc[:,'gene_name']
M_MD = M_MD.loc[:,M_MD.columns!='gene_name'] # Shape (19938, 77)

#M_MD = M_MD.drop_duplicates() # Shape (19292, 77)

M_MD.dropna(inplace=True) # Shape (19292, 77)

# Drop if any gene that contains 0 read of TPM (5063 genes are dropped, still have 6706 in common with PD)
M_MD = M_MD.loc[~(M_MD==0).any(axis=1)] # Shape (14229, 77)

# Sort the column names in the dataframe
M_MD = M_MD.reindex(sorted(M_MD.columns), axis=1) # Shape (14229, 77)

#### Normalization

In [18]:
# Visualize the processed data with boxplot and histogram
M_PD_norm = Normalization(M_PD)
M_MD_norm = Normalization(M_MD)

#### Quantile transform

In [19]:
#Quantile transformation
M_PD_qt = pd.DataFrame(QuantileTransformer(output_distribution='normal',random_state=42).fit_transform(M_PD_norm), index=M_PD_norm.index, columns=M_PD_norm.columns)
M_MD_qt = pd.DataFrame(QuantileTransformer(output_distribution='normal',random_state=43).fit_transform(M_MD_norm), index=M_MD_norm.index, columns=M_MD_norm.columns)

#### Match gene name in proteome and transcriptome

In [20]:
common_genes = M_PD_qt.index.intersection(M_MD_qt.index)
M_PD_qt = M_PD_qt.loc[common_genes]
M_MD_qt = M_MD_qt.loc[common_genes]

#### Inferencing

In [21]:
# Ensure reproducibility
set_seed(44)

# Synthetic tumors using gaussian KDE
M_PD_kde, M_MD_kde = Gaussian_KDE(M_PD_qt, M_MD_qt)

Synthetic data generated


#### Filter out genes that has labels

In [22]:
# Open the labels data
LFP = 'markers.txt'
LD = pd.read_csv(filepath_or_buffer=label_path+LFP,sep='\t')

# Data set wrangling
LD.index = LD.loc[:,'Protein']
LD = LD.loc[:,LD.columns!='Protein']

# Remove unclassified class
NotUnclassInd = LD.loc[:,'Localization'] != 'Unclassified'
LD = LD.loc[NotUnclassInd,:]

In [23]:
# Keep only genes (rows) are presented in proteome, mRNA and localization data sets
IntersectingGenes = [value for value in M_PD_kde.index if ((value in M_MD_kde.index) & (value in LD.index))]
M_PD_kde = M_PD_kde.loc[IntersectingGenes,:]
M_MD_kde = M_MD_kde.loc[IntersectingGenes,:]
LD = LD.loc[IntersectingGenes,:]

# Sanity check for the number of genes in each dataframe
print('Mertin')
print(len(M_PD_kde.index))
print(len(M_MD_kde.index))
print(len(LD.index))

Mertin
2355
2355
2355


In [24]:
# Count the number of genes in each localization class
LocalizationCounts = LD.loc[:,'Localization'].value_counts()
print('Number of genes in each localization class:')
print(LocalizationCounts)

Number of genes in each localization class:
Localization
Cytosol         982
Secretory       580
Nucleus         550
Mitochondria    243
Name: count, dtype: int64


### Preparing data for LCSCC Tumor

#### Raw data processing

In [25]:
# Open the proteomics data and only keep genes (rows) that are fully quantified
lung_tumor_PFP = 'lcscc tumor prot model use.csv' # MSFragger search, abundances are ratio of channel to pool, no transformation
lung_tumor_PD = pd.read_csv(raw_data_path+lung_tumor_PFP, index_col=0) # Shape (13506, 108)

# Remove NA
lung_tumor_PD.dropna(inplace=True) # Shape (9051, 108)

# Check if any index is duplicated, if so, average the duplicates
lung_tumor_PD = lung_tumor_PD.groupby(lung_tumor_PD.index).mean()

# Sort the column names in the dataframe
lung_tumor_PD = lung_tumor_PD.reindex(sorted(lung_tumor_PD.columns), axis=1) # Shape (8481, 108)

In [26]:
# Open the mRNA data and only keep genes (rows) that are fully quantified
lung_tumor_MFP = 'lcscc tumor mrna model use.csv' # TPM data, untransformed
lung_tumor_MD = pd.read_csv(raw_data_path+lung_tumor_MFP, index_col=0) # Shape (19938, 108)

# Data set wrangling
lung_tumor_MD.dropna(inplace=True) # Shape (19938, 108)

# Drop if any gene that contains 0 read of TPM
lung_tumor_MD = lung_tumor_MD.loc[~(lung_tumor_MD==0).any(axis=1)] # Shape (15387, 108)

# Check if any index is duplicated, if so, average the duplicates
lung_tumor_MD = lung_tumor_MD.groupby(lung_tumor_MD.index).mean()

# Sort the column names in the dataframe
lung_tumor_MD = lung_tumor_MD.reindex(sorted(lung_tumor_MD.columns), axis=1) # Shape (15387, 108)

#### Normalization

In [27]:
# Visualize the processed data with boxplot and histogram
lung_tumor_PD_norm = Normalization(lung_tumor_PD)
lung_tumor_MD_norm = Normalization(lung_tumor_MD)

#### Quantile transform

In [28]:
# Quantile transform each tumor
lung_tumor_PD_qt = pd.DataFrame(QuantileTransformer(output_distribution='normal',random_state=42).fit_transform(lung_tumor_PD_norm), index=lung_tumor_PD_norm.index, columns=lung_tumor_PD_norm.columns)
lung_tumor_MD_qt = pd.DataFrame(QuantileTransformer(output_distribution='normal',random_state=43).fit_transform(lung_tumor_MD_norm), index=lung_tumor_MD_norm.index, columns=lung_tumor_MD_norm.columns)

#### Match gene name in proteome and transcriptome

In [29]:
common_genes = lung_tumor_PD_qt.index.intersection(lung_tumor_MD_qt.index)
lung_tumor_PD_qt = lung_tumor_PD_qt.loc[common_genes]
lung_tumor_MD_qt = lung_tumor_MD_qt.loc[common_genes]

#### Inferencing

In [30]:
# Ensure reproducibility
set_seed(46)

# Synthetic tumors using gaussian KDE
lung_tumor_PD_kde, lung_tumor_MD_kde = Gaussian_KDE(lung_tumor_PD_qt, lung_tumor_MD_qt)

Synthetic data generated


#### Filter out genes that has labels

In [31]:
# Open the labels data
LFP = 'markers.txt'
lung_tumor_LD = pd.read_csv(filepath_or_buffer=label_path+LFP,sep='\t')

# Data set wrangling
lung_tumor_LD.index = lung_tumor_LD.loc[:,'Protein']
lung_tumor_LD = lung_tumor_LD.loc[:,lung_tumor_LD.columns!='Protein']

# Remove unclassified class
NotUnclassInd = lung_tumor_LD.loc[:,'Localization'] != 'Unclassified'
lung_tumor_LD = lung_tumor_LD.loc[NotUnclassInd,:]

In [32]:
# Keep only genes (rows) are presented in proteome, mRNA and localization data sets
IntersectingGenes = [value for value in lung_tumor_PD_kde.index if ((value in lung_tumor_MD_kde.index) & (value in lung_tumor_LD.index))]
lung_tumor_PD_kde = lung_tumor_PD_kde.loc[IntersectingGenes,:]
lung_tumor_MD_kde = lung_tumor_MD_kde.loc[IntersectingGenes,:]
lung_tumor_LD = lung_tumor_LD.loc[IntersectingGenes,:]

# Sanity check for the number of genes in each dataframe
print('LCSCC Tumor')
print(len(lung_tumor_PD_kde.index))
print(len(lung_tumor_MD_kde.index))
print(len(lung_tumor_LD.index))

LCSCC Tumor
2788
2788
2788


In [33]:
# Count the number of genes in each localization class
LocalizationCounts = lung_tumor_LD.loc[:,'Localization'].value_counts()
print('Number of genes in each localization class:')
print(LocalizationCounts)

Number of genes in each localization class:
Localization
Cytosol         1035
Secretory        864
Nucleus          624
Mitochondria     265
Name: count, dtype: int64


### Preparing data for LCSCC NAT

#### Raw data processing

In [34]:
# Open the proteomics data and only keep genes (rows) that are fully quantified
lung_nat_PFP = 'lcscc nat prot model use.csv' # MSFragger search, abundances are ratio of channel to pool, no transformation
lung_nat_PD = pd.read_csv(raw_data_path+lung_nat_PFP, index_col=0) # Shape (13506, 94)

# Remove NA
lung_nat_PD.dropna(inplace=True) # Shape (9047, 94)

# Check if any index is duplicated, if so, average the duplicates
lung_nat_PD = lung_nat_PD.groupby(lung_nat_PD.index).mean() # Shape (9041, 94)

# Sort the column names in the dataframe
lung_nat_PD = lung_nat_PD.reindex(sorted(lung_nat_PD.columns), axis=1) # Shape (9041, 94)

In [35]:
# Open the mRNA data and only keep genes (rows) that are fully quantified
lung_nat_MFP = 'lcscc nat mrna model use.csv' # TPM data, untransformed
lung_nat_MD = pd.read_csv(raw_data_path+lung_nat_MFP, index_col=0) # Shape (19938, 94)

# Data set wrangling
lung_nat_MD.dropna(inplace=True) # Shape (19938, 94)

# Drop if any gene that contains 0 read of TPM
lung_nat_MD = lung_nat_MD.loc[~(lung_nat_MD==0).any(axis=1)] # Shape (16095, 94)

# Check if any index is duplicated, if so, average the duplicates
lung_nat_MD = lung_nat_MD.groupby(lung_nat_MD.index).mean() # Shape (16095, 94)

# Sort the column names in the dataframe
lung_nat_MD = lung_nat_MD.reindex(sorted(lung_nat_MD.columns), axis=1) # Shape (16095, 94)

#### Normalization

In [36]:
# Visualize the processed data with boxplot and histogram
lung_nat_PD_norm = Normalization(lung_nat_PD)
lung_nat_MD_norm = Normalization(lung_nat_MD)

#### Quantile transform

In [37]:
# Quantile transform each tumor
lung_nat_PD_qt = pd.DataFrame(QuantileTransformer(output_distribution='normal',random_state=42).fit_transform(lung_nat_PD_norm), index=lung_nat_PD_norm.index, columns=lung_nat_PD_norm.columns)
lung_nat_MD_qt = pd.DataFrame(QuantileTransformer(output_distribution='normal',random_state=43).fit_transform(lung_nat_MD_norm), index=lung_nat_MD_norm.index, columns=lung_nat_MD_norm.columns)

#### Match gene name in proteome and transcriptome

In [38]:
common_genes = lung_nat_PD_qt.index.intersection(lung_nat_MD_qt.index)
lung_nat_PD_qt = lung_nat_PD_qt.loc[common_genes]
lung_nat_MD_qt = lung_nat_MD_qt.loc[common_genes]

#### Inferencing

In [39]:
# Ensure reproducibility
set_seed(48)

# Synthetic tumors using gaussian KDE
lung_nat_PD_kde, lung_nat_MD_kde = Gaussian_KDE(lung_nat_PD_qt, lung_nat_MD_qt)

Synthetic data generated


#### Filter out genes that has labels

In [40]:
# Open the labels data
LFP = 'markers.txt'
lung_nat_LD = pd.read_csv(filepath_or_buffer=label_path+LFP,sep='\t')

# Data set wrangling
lung_nat_LD.index = lung_nat_LD.loc[:,'Protein']
lung_nat_LD = lung_nat_LD.loc[:,lung_nat_LD.columns!='Protein']

# Remove unclassified class
NotUnclassInd = lung_nat_LD.loc[:,'Localization'] != 'Unclassified'
lung_nat_LD = lung_nat_LD.loc[NotUnclassInd,:]

In [41]:
# Keep only genes (rows) are presented in proteome, mRNA and localization data sets
IntersectingGenes = [value for value in lung_nat_PD_kde.index if ((value in lung_nat_MD_kde.index) & (value in lung_nat_LD.index))]
lung_nat_PD_kde = lung_nat_PD_kde.loc[IntersectingGenes,:]
lung_nat_MD_kde = lung_nat_MD_kde.loc[IntersectingGenes,:]
lung_nat_LD = lung_nat_LD.loc[IntersectingGenes,:]

# Sanity check for the number of genes in each dataframe
print('LCSCC nat')
print(len(lung_nat_PD_kde.index))
print(len(lung_nat_MD_kde.index))
print(len(lung_nat_LD.index))

LCSCC nat
2789
2789
2789


In [42]:
# Count the number of genes in each localization class
LocalizationCounts = lung_nat_LD.loc[:,'Localization'].value_counts()
print('Number of genes in each localization class:')
print(LocalizationCounts)

Number of genes in each localization class:
Localization
Cytosol         1036
Secretory        864
Nucleus          624
Mitochondria     265
Name: count, dtype: int64


### Preparing data for Gliomas

#### Raw data processing

In [43]:
# Open the proteomics data and only keep genes (rows) that are fully quantified
glioma_tumor_PFP = 'glioma prot model use.csv' # MSFragger search, abundances are ratio of channel to pool, no transformation
glioma_tumor_PD = pd.read_csv(raw_data_path+glioma_tumor_PFP, index_col=0) # Shape (13295, 150)

# Remove columns that contain 'Disqualified' 
glioma_tumor_PD = glioma_tumor_PD.loc[:,~glioma_tumor_PD.columns.str.contains('Disqualified')] # Shape (13295, 148)

# Remove columns that contain 'Recurrent'
glioma_tumor_PD = glioma_tumor_PD.loc[:,~glioma_tumor_PD.columns.str.contains('Recurrent')] # Shape (13295, 123)

# Remove columns that contain 'Metastatic'
glioma_tumor_PD = glioma_tumor_PD.loc[:,~glioma_tumor_PD.columns.str.contains('Metastatic')] # Shape (13295, 111)

# Remove columns that contain 'Solid-Tissue-Normal'
glioma_tumor_PD = glioma_tumor_PD.loc[:,~glioma_tumor_PD.columns.str.contains('Solid-Tissue-Normal')] # Shape (13295, 103)

# Check if how many columns contain 'ref2'
glioma_tumor_PD = glioma_tumor_PD.loc[:,~glioma_tumor_PD.columns.str.contains('ref2')] # Shape (13295, 102)

# Modify the column names to remove '-Primary-Tumor'
glioma_tumor_PD.columns = glioma_tumor_PD.columns.str.replace('-Primary-Tumor', '') # Shape (13295, 102)

# Remove NA
glioma_tumor_PD.dropna(inplace=True) # Shape (8589, 102)

# Check if any index is duplicated, if so, average the duplicates
glioma_tumor_PD = glioma_tumor_PD.groupby(glioma_tumor_PD.index).mean() # Shape (8580, 102)

# Sort the column names in the dataframe
glioma_tumor_PD = glioma_tumor_PD.reindex(sorted(glioma_tumor_PD.columns), axis=1) # Shape (8580, 102)

In [44]:
# Open the mRNA data and only keep genes (rows) that are fully quantified
glioma_tumor_MFP = 'glioma mrna model use.csv' # TPM data, untransformed
glioma_tumor_MD = pd.read_csv(raw_data_path+glioma_tumor_MFP, index_col=0) # Shape (19938, 203)

# Data set wrangling
glioma_tumor_MD.dropna(inplace=True) # Shape (19938, 203)

# Drop if any gene that contains 0 read of TPM
glioma_tumor_MD = glioma_tumor_MD.loc[~(glioma_tumor_MD==0).any(axis=1)] # Shape (15010, 203)

# Check if any index is duplicated, if so, average the duplicates
glioma_tumor_MD = glioma_tumor_MD.groupby(glioma_tumor_MD.index).mean() # Shape (15010, 203)

# Sort the column names in the dataframe
glioma_tumor_MD = glioma_tumor_MD.reindex(sorted(glioma_tumor_MD.columns), axis=1) # Shape (15010, 203)

In [45]:
# get the column names that are common in proteome and mRNA data sets
common_col = [col for col in glioma_tumor_PD.columns if col in glioma_tumor_MD.columns]

# filter the columns in both data sets
glioma_tumor_PD = glioma_tumor_PD.loc[:,common_col] # shape (8580, 99)
glioma_tumor_MD = glioma_tumor_MD.loc[:,common_col] # shape (15010, 99)

#### Normalization

In [46]:
# Visualize the processed data with boxplot and histogram
glioma_tumor_PD_norm = Normalization(glioma_tumor_PD)
glioma_tumor_MD_norm = Normalization(glioma_tumor_MD)

#### Quantile transfrom

In [47]:
# Quantile transform each tumor
glioma_tumor_PD_qt = pd.DataFrame(QuantileTransformer(output_distribution='normal',random_state=42).fit_transform(glioma_tumor_PD_norm), index=glioma_tumor_PD_norm.index, columns=glioma_tumor_PD_norm.columns)
glioma_tumor_MD_qt = pd.DataFrame(QuantileTransformer(output_distribution='normal',random_state=43).fit_transform(glioma_tumor_MD_norm), index=glioma_tumor_MD_norm.index, columns=glioma_tumor_MD_norm.columns)

#### Match gene name in proteome and transcriptome

In [48]:
common_genes = glioma_tumor_PD_qt.index.intersection(glioma_tumor_MD_qt.index)
glioma_tumor_PD_qt = glioma_tumor_PD_qt.loc[common_genes]
glioma_tumor_MD_qt = glioma_tumor_MD_qt.loc[common_genes]

#### Inferencing

In [49]:
# Ensure reproducibility
set_seed(50)

# Synthetic tumors using gaussian KDE
glioma_tumor_PD_kde, glioma_tumor_MD_kde = Gaussian_KDE(glioma_tumor_PD_qt, glioma_tumor_MD_qt)

Synthetic data generated


#### Filter out genes that has labels

In [50]:
# Open the labels data
LFP = 'markers.txt'
glioma_tumor_LD = pd.read_csv(filepath_or_buffer=label_path+LFP,sep='\t')

# Data set wrangling
glioma_tumor_LD.index = glioma_tumor_LD.loc[:,'Protein']
glioma_tumor_LD = glioma_tumor_LD.loc[:,glioma_tumor_LD.columns!='Protein']

# Remove unclassified class
NotUnclassInd = glioma_tumor_LD.loc[:,'Localization'] != 'Unclassified'
glioma_tumor_LD = glioma_tumor_LD.loc[NotUnclassInd,:]

In [51]:
# Keep only genes (rows) are presented in proteome, mRNA and localization data sets
IntersectingGenes = [value for value in glioma_tumor_PD_kde.index if ((value in glioma_tumor_MD_kde.index) & (value in glioma_tumor_LD.index))]
glioma_tumor_PD_kde = glioma_tumor_PD_kde.loc[IntersectingGenes,:]
glioma_tumor_MD_kde = glioma_tumor_MD_kde.loc[IntersectingGenes,:]
glioma_tumor_LD = glioma_tumor_LD.loc[IntersectingGenes,:]

# Sanity check for the number of genes in each dataframe
print('glioma tumor')
print(len(glioma_tumor_PD_kde.index))
print(len(glioma_tumor_MD_kde.index))
print(len(glioma_tumor_LD.index))

glioma tumor
2679
2679
2679


In [52]:
# count the number of different labels
print(glioma_tumor_LD['Localization'].value_counts())

Localization
Cytosol         1013
Secretory        847
Nucleus          553
Mitochondria     266
Name: count, dtype: int64


### Export

#### Krug

In [ ]:
# Export
PD_kde.to_csv(Path(output_path + 'K_PD_synthetic_kde_june.csv'))
MD_kde.to_csv(Path(output_path + 'K_MD_synthetic_kde_june.csv'))

#### Mertins

In [ ]:
# Export
M_PD_kde.to_csv(Path(output_path + 'M_PD_synthetic_kde_june.csv'))
M_MD_kde.to_csv(Path(output_path + 'M_MD_synthetic_kde_june.csv'))

#### LCSCC Tumor

In [ ]:
# Export
lung_tumor_PD_kde.to_csv(Path(output_path + 'lung_tumor_PD_synthetic_kde_june.csv'))
lung_tumor_MD_kde.to_csv(Path(output_path + 'lung_tumor_MD_synthetic_kde_june.csv'))

#### LCSCC NAT

In [ ]:
# Export
lung_nat_PD_kde.to_csv(Path(output_path + 'lung_nat_PD_synthetic_kde_june.csv'))
lung_nat_MD_kde.to_csv(Path(output_path + 'lung_nat_MD_synthetic_kde_june.csv'))

#### Brain glioma

In [ ]:
# Export
glioma_tumor_PD_kde.to_csv(Path(output_path + 'glioma_tumor_PD_synthetic_kde_june.csv'))
glioma_tumor_MD_kde.to_csv(Path(output_path + 'glioma_tumor_MD_synthetic_kde_june.csv'))